In [1]:
import numpy as np
import pandas as pd
import json
import os
os.environ["OMP_NUM_THREADS"] = '1'


from langchain.prompts import PromptTemplate
from llama_index.embeddings.langchain import LangchainEmbedding
from langchain.embeddings import HuggingFaceEmbeddings

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
#from umap import UMAP

from umap import umap_ as UMAP


from sklearn.cluster import KMeans
from matplotlib import pyplot as plt

import umap